# Traffic sign detection and classification

In [14]:
from xml.dom import minidom
from os import walk
import pandas as pd
import cv2 as cv
import numpy as np
from sklearn import metrics


In [2]:
filenames = next(walk("res/annotations"), (None, None, []))[2]  # [] if no file

signs = []

for annotation in filenames:
    # parse an xml file by name
    file = minidom.parse("res/annotations/" + annotation)

    #use getElementsByTagName() to get tag
    path = "res/images/" + file.getElementsByTagName('filename')[0].firstChild.data
    filename = file.getElementsByTagName('filename')[0].firstChild.data
    name = file.getElementsByTagName('name')[0].firstChild.data
    # truncated = file.getElementsByTagName('truncated')[0].firstChild.data
    # occluded = file.getElementsByTagName('occluded')[0].firstChild.data
    # difficult = file.getElementsByTagName('difficult')[0].firstChild.data

    if name == "trafficlight":
        continue

    signs.append([filename, name, path])

df = pd.DataFrame(signs, columns=['filename', 'name', 'path'])

In [3]:
def condition_classes(s):
    if s['name'] == 'speedlimit':
        return 0
    elif s["name"] == 'crosswalk':
        return 1
    elif s["name"] == "stop":
        return 2


df["class"] = df.apply(condition_classes, axis=1)

In [4]:
def concatenate_and_write(row):
    img = cv.imread(row.path)
    hist = cv.imread("output/histogram/" + row.filename)
    segm = cv.imread("output/segmentation/" + row.filename)
    post = cv.imread("output/post_processing/" + row.filename)

    vis = np.concatenate((img, hist, segm, post), axis=1)

    cv.imwrite("output/concatenated/" + row.filename, vis)

## Step 1 - Histogram equalization

TODO - Need to improve the histogram equalization

In [5]:
def apply_histogram_equalization(row):
    img = cv.imread(row.path)
    # convert image from RGB to HSV
    img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    # Histogram equalisation on the V-channel
    img_hsv[:, :, 2] = cv.equalizeHist(img_hsv[:, :, 2])
    # convert image back from HSV to RGB
    out = cv.cvtColor(img_hsv, cv.COLOR_HSV2BGR)

    cv.imwrite("output/histogram/" + row.filename, out)

In [6]:
df.apply(apply_histogram_equalization, axis=1);

## Step 2 - Segmentation by Color

In [7]:
def apply_segmentation(row):
    img = cv.imread(row.path)
    # TODO - work on histogram equalization
    # img_hist = cv.imread("output/histogram/" + row.filename)
    img_hist = img
    img_hsv = cv.cvtColor(img_hist, cv.COLOR_BGR2HSV)

    lower_red_m1 = (0, 70, 60)
    upper_red_m1 = (10, 255, 255)

    lower_red_m2 = (170, 70, 60)
    upper_red_m2 = (180, 255, 255)

    lower_blue_m3 = (94, 127, 20)
    upper_blue_m3 = (126, 255, 200)

    mask1 = cv.inRange(img_hsv, lower_red_m1, upper_red_m1)
    mask2 = cv.inRange(img_hsv, lower_red_m2, upper_red_m2)
    mask3 = cv.inRange(img_hsv, lower_blue_m3, upper_blue_m3)

    mask = mask1 + mask2 + mask3

    # out = cv.bitwise_and(img_hist, img_hist, mask=mask)

    cv.imwrite("output/segmentation/" + row.filename, mask)

In [8]:
df.apply(apply_segmentation, axis=1);

## Step 3 - Post-Processing

In [5]:
def apply_post_processing(row):
    img = cv.imread(row.path)
    segm_img = cv.imread("output/segmentation/" + row.filename, cv.IMREAD_GRAYSCALE)

    # apply median filter to remove noise
    out = cv.medianBlur(segm_img, 5)
    rows, cols = out.shape

    # Taking a matrix of size 5 as the kernel
    kernel = np.ones((5, 5), np.uint8)

    # morphological operations
    out = cv.dilate(out, kernel, iterations=1)
    out = cv.erode(out, kernel, iterations=1)

    # remove small and weird objects
    contours, hierarchy = cv.findContours(out, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        x, y, w, h = cv.boundingRect(contour)
        aspect_ratio = float(w) / h
        if cv.contourArea(contour) < 1 / 1500.0 * rows * cols and (aspect_ratio > 0.5 or aspect_ratio < 1.3):
            out = cv.fillPoly(out, pts=contour, color=(0, 0, 0))

    mask = np.full(img.shape, 0, "uint8")
    contours, hierarchies = cv.findContours(out, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
    for cnt in contours:
        cv.drawContours(mask, [cnt], -1, (255, 255, 255), -1)

    mask = cv.cvtColor(mask, cv.COLOR_BGR2GRAY)
    # morphological operations
    out = cv.erode(mask, kernel, iterations=1)
    out = cv.dilate(mask, kernel, iterations=1)

    cv.imwrite("output/post_processing/" + row.filename, out)

In [22]:
df.apply(apply_post_processing, axis=1);

In [ ]:
df.apply(concatenate_and_write, axis=1);

In [5]:
def shape_recognition(row):
    img = cv.imread(row.path)
    processed = cv.imread("output/post_processing/" + row.filename, 0)

    _, thresh = cv.threshold(processed, 240, 255, cv.CHAIN_APPROX_NONE)
    contours, _ = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)

    contours = sorted(contours, key=lambda x: -cv.contourArea(x))[:5]

    shapes = []

    for contour in contours:
        if float(cv.contourArea(contour) / (img.shape[0]*img.shape[1])) >= 0.95:
            continue
        approx = cv.approxPolyDP(contour, 0.01*cv.arcLength(contour, True), True)
        if len(approx) == 4:
            shapes.append(("rectangle", cv.contourArea(contour), (cv.boundingRect(contour))))
        elif len(approx) == 8:
            shapes.append(("octagon", cv.contourArea(contour), (cv.boundingRect(contour))))
        elif len(approx) > 8:
            shapes.append(("circle", cv.contourArea(contour), (cv.boundingRect(contour))))

    return shapes

In [8]:
df["shapes"] = df.apply(shape_recognition, axis=1);

In [12]:
def classify(row):
    shapes = row.shapes
    if len(shapes) == 0:
        return -1

    shape = shapes[0]

    if shape[0] == 'circle':
        return 0
    elif shape[0] == 'rectangle':
        return 1
    elif shape[0] == 'octagon':
        return 2

    return -1

df["classification"] = df.apply(classify, axis=1);

In [18]:
accuracy = metrics.accuracy_score(df["class"], df["classification"])

print("Accuracy: {:.02f}%".format(accuracy*100))

Accuracy: 67.28%
